# coreKanren in Python


In [1]:
from copy import deepcopy


In [2]:
def var(name):
    return ('_var', name)

def nvar(v):
    assert(v[0] == '_var')
    return v[1]

def is_var(v):
    try:
        return v[0] == '_var'
    except:
        return False


In [3]:
def assoc(lhs, rhs):
    return ('_assoc', lhs, rhs)

def lhs(asso):
    assert(asso[0] == '_assoc')
    return asso[1]


def rhs(asso):
    assert(asso[0] == '_assoc')
    return asso[2]


In [4]:
def subs(assocl):
    for i in assocl:
        assert(i[0] == '_assoc')
    return ('_s', assocl)


In [5]:
def assq(v, s):
    assert(s[0] == '_s')
    assocl = s[1]
    for a in assocl:
        if nvar(lhs(a)) == nvar(v):
            return a
    return None


In [6]:
def walk(v, s, debug=False):
    walked = set()
    def _walk(v, s):
        assert(s[0] == '_s')
        assert(not str(v) in walked), "Cyclic Walk"
        walked.add(str(v))
        ret = None
        if is_var(v):
            a = assq(v, s)
            if a != None:
                ret = _walk(rhs(a), s)
            else:
                ret = v
        else:
            ret = v
        if (debug):
            print("WALK", v, s, "=>", ret)
        return ret
    return _walk(v, s)

In [7]:
# (walk u ( (x . b) (w . (x e x)) (u . w) )

test_s = subs([assoc(var('x'), 'b'), assoc(var('w'), [var('x'), 'e', var('x')]), assoc(var('u'), var('w'))])
walk(var('u'), test_s, debug=True)

WALK [('_var', 'x'), 'e', ('_var', 'x')] ('_s', [('_assoc', ('_var', 'x'), 'b'), ('_assoc', ('_var', 'w'), [('_var', 'x'), 'e', ('_var', 'x')]), ('_assoc', ('_var', 'u'), ('_var', 'w'))]) => [('_var', 'x'), 'e', ('_var', 'x')]
WALK ('_var', 'w') ('_s', [('_assoc', ('_var', 'x'), 'b'), ('_assoc', ('_var', 'w'), [('_var', 'x'), 'e', ('_var', 'x')]), ('_assoc', ('_var', 'u'), ('_var', 'w'))]) => [('_var', 'x'), 'e', ('_var', 'x')]
WALK ('_var', 'u') ('_s', [('_assoc', ('_var', 'x'), 'b'), ('_assoc', ('_var', 'w'), [('_var', 'x'), 'e', ('_var', 'x')]), ('_assoc', ('_var', 'u'), ('_var', 'w'))]) => [('_var', 'x'), 'e', ('_var', 'x')]


[('_var', 'x'), 'e', ('_var', 'x')]

In [8]:
def ext_s(x, v, s):
    assert(s[0] == '_s')
    assert(x[0] == '_var')
    a = assoc(x, v)
    assocl = s[1] + [a]
    return subs(assocl)


In [9]:
# (walk x (ext-s x y ( (z . x) (y . z) )
test_s = ext_s(var('x'), var('y'), subs(
    [assoc(var('z'), var('x')), assoc(var('y'), var('z'))]))
walk(var('x'), test_s, debug = True)


AssertionError: Cyclic Walk

In [10]:
def is_eq(a, b):
    if type(a) != type(b):
        return False
    if type(a) == int:
        return a == b
    elif type(a) == str:
        return a == b
    elif len(a) != len(b):
        return False
    flag = True
    for i in range(len(a)):
        if is_eq(a[i], b[i]) == False:
            flag = False
    return flag

In [11]:
is_eq(subs([assoc(var('x'), 'b'), assoc(var('w'), [var('x'), 'e', var('x')]), assoc(var('u'), var('w'))]), subs([assoc(var('x'), 'b'), assoc(var('w'), [var('x'), 'e', var('x')]), assoc(var('u'), var('w'))]))

True

In [12]:
is_eq(ext_s(var('x'), var('y'), subs([assoc(var('z'), var('x')), assoc(var('y'), var('z'))])),subs([assoc(var('x'), 'b'), assoc(var('w'), [var('x'), 'e', var('x')]), assoc(var('u'), var('w'))]))

False

In [13]:
def unify(v, w, s, debug=False):
    v = walk(v, s)
    w = walk(w, s)
    ret = None
    if is_eq(v, w):
        ret = s
    elif is_var(v):
        ret = ext_s(v, w, s)
    elif is_var(w):
        ret = ext_s(w, v, s)
    elif type(v) == list and type(w) == list:
        tu = unify(v[0], w[0], s)
        if tu != None:
            ret = unify(v[1:], w[1:], tu)
        else:
            ret = None
    else:
        ret = None
    if debug:
        print("UNIFY", v, w, s, "=>", ret)
    return ret


In [14]:
test_v = [[1, var('x'), 3], var('z')]
test_w = [[1, 5, var('y')], "Hello"]
print(unify(test_v, test_w, subs([])))

('_s', [('_assoc', ('_var', 'x'), 5), ('_assoc', ('_var', 'y'), 3), ('_assoc', ('_var', 'z'), 'Hello')])


In [15]:
def deep_walk(v, s, debug=False):
    v = walk(v, s)
    ret = None
    if is_var(v):
        ret = v
    elif type(v) == list:
        ret = []
        for i in v:
            ret.append(deep_walk(i, s))
    else:
        ret = v
    if debug:
        print("WALK*", v, s, "=>", ret)
    return ret

In [16]:
# (walk* u ( (x . b) (w . (x e x)) (u . w) )

test_s = subs([assoc(var('x'), 'b'), assoc(var('w'), [var('x'), 'e', var('x')]), assoc(var('u'), var('w'))])
deep_walk(var('u'), test_s, debug=True)

WALK* [('_var', 'x'), 'e', ('_var', 'x')] ('_s', [('_assoc', ('_var', 'x'), 'b'), ('_assoc', ('_var', 'w'), [('_var', 'x'), 'e', ('_var', 'x')]), ('_assoc', ('_var', 'u'), ('_var', 'w'))]) => ['b', 'e', 'b']


['b', 'e', 'b']

In [17]:
def reify_name(n):
    return "_.{}".format(n)


def reify_s(v, s, debug=False):
    v = walk(v, s)
    ret = None
    if is_var(v):
        ret = ext_s(v, reify_name(len(s[1])), s)
    elif type(v) == list:
        if len(v) > 0:
            ret = reify_s(v[1:], reify_s(v[0], s))
        else:
            ret = s
    else:
        ret = s
    if debug:
        print("REIFY-S", v, s, "=>", ret)
    return ret


In [18]:
test_r = [var('w'), var('x'), var('y')]
test_s = reify_s(test_r, subs([]), debug=True)
test_r = deep_walk(test_r, test_s)
test_s = reify_s(test_r, subs([]), debug=True)
deep_walk(test_r, test_s)

REIFY-S [('_var', 'w'), ('_var', 'x'), ('_var', 'y')] ('_s', []) => ('_s', [('_assoc', ('_var', 'w'), '_.0'), ('_assoc', ('_var', 'x'), '_.1'), ('_assoc', ('_var', 'y'), '_.2')])
REIFY-S ['_.0', '_.1', '_.2'] ('_s', []) => ('_s', [])


['_.0', '_.1', '_.2']

In [19]:
test_r = [var('u'), [var('v'), [var('w'), var('x')], var('y')], var('x')]
test_s = reify_s(test_r, subs([]), debug=True)
deep_walk(test_r, test_s)

REIFY-S [('_var', 'u'), [('_var', 'v'), [('_var', 'w'), ('_var', 'x')], ('_var', 'y')], ('_var', 'x')] ('_s', []) => ('_s', [('_assoc', ('_var', 'u'), '_.0'), ('_assoc', ('_var', 'v'), '_.1'), ('_assoc', ('_var', 'w'), '_.2'), ('_assoc', ('_var', 'x'), '_.3'), ('_assoc', ('_var', 'y'), '_.4')])


['_.0', ['_.1', ['_.2', '_.3'], '_.4'], '_.3']

In [20]:
test_s = subs([assoc(var('y'), [var('z'), var('w'), 'c', var('w')]), assoc(var('x'), var('y')), assoc(var('z'), 'a')])
test_r = deep_walk(var('x'), test_s, debug=True)
test_s1 = reify_s(test_r, subs([]), debug=True)
deep_walk(test_r, test_s1)

WALK* [('_var', 'z'), ('_var', 'w'), 'c', ('_var', 'w')] ('_s', [('_assoc', ('_var', 'y'), [('_var', 'z'), ('_var', 'w'), 'c', ('_var', 'w')]), ('_assoc', ('_var', 'x'), ('_var', 'y')), ('_assoc', ('_var', 'z'), 'a')]) => ['a', ('_var', 'w'), 'c', ('_var', 'w')]
REIFY-S ['a', ('_var', 'w'), 'c', ('_var', 'w')] ('_s', []) => ('_s', [('_assoc', ('_var', 'w'), '_.0')])


['a', '_.0', 'c', '_.0']

In [21]:
def reify(v, debug=False):
    s = reify_s(v, subs([]), debug=debug)
    ret = deep_walk(v, s)
    if debug:
        print("REIFY", v, "=>", ret)
    return ret

In [22]:
test_s = subs([assoc(var('y'), [var('z'), var('w'), 'c', var('w')]), assoc(var('x'), var('y')), assoc(var('z'), 'a')])
reify(deep_walk(var('x'), test_s))

['a', '_.0', 'c', '_.0']

In [24]:
def occurs_d(x, v, s, debug=False):
    ret = None
    v = walk(v, s, debug=debug)
    if is_var(v):
        ret = is_eq(v, x)
    elif type(v) == list:
        flag = False
        for i in v:
            if occurs_d(x, i, s, debug=debug):
                flag = True
                break
        ret = flag
    if debug:
        print("OCCURS√", x, v, s, "=>", ret)
    return ret


def ext_sd(x, v, s):
    if occurs_d(x, v, s):
        return None
    else:
        return ext_s(x, v, s)


In [25]:
def unify_d(v, w, s, debug=False):
    v = walk(v, s)
    w = walk(w, s)
    ret = None
    if is_eq(v, w):
        ret = s
    elif is_var(v):
        ret = ext_sd(v, w, s)
    elif is_var(w):
        ret = ext_sd(w, v, s)
    elif type(v) == list and type(w) == list:
        tu = unify_d(v[0], w[0], s)
        if tu != None:
            ret = unify_d(v[1:], w[1:], tu)
        else:
            ret = None
    else:
        ret = None
    if debug:
        print("UNIFY√", v, w, s, "=>", ret)
    return ret


In [26]:
test_v = [[1, var('x'), 3], var('z')]
test_w = [[1, 5, var('y')], "Hello"]
print(unify_d(test_v, test_w, subs([])))

('_s', [('_assoc', ('_var', 'x'), 5), ('_assoc', ('_var', 'y'), 3), ('_assoc', ('_var', 'z'), 'Hello')])
